# Predicción del daño sísmico en Gorkha, 2015

En Abril de 2015, Gorkha, ciudad de 32.000 habitantes que se encuentra en Nepal, fue víctima de un terremoto de magnitud 7.8. Este desastre natural produjo unas pérdidas de 10 billones de dólares, lo cual representa la mitad del PIB de Nepal, por lo que el gobierno tuvo que entrar en largos y costosos procesos de reparación.

Dentro de este proceso de reconstrucción y análisis de los daños causados, se tuvo que identificar cuál fue el nivel de destrucción de las infraestructras. Y es en este punto donde juega un papel importante el análisis de datos, y concretamente el trabajo de un Data Scientist, ya que trataremos de analizar el Dataset Ritcher’s Predictor: Modeling Earthquake Damage. En este caso, el dataset estudiado cuenta con los datos de las infraestructuras dañadas de Gorkha y  todas sus características que más tarde pasaremos a analizar detalladamente. De esta manera, para determinar qué nivel de reparación necesita una infraestructura dañada, es necesario catalogar el nivel de destrucción de esta después de la catástrofe. Una vez hecho esto, es posible confeccionar un modelo capaz de predecir cual sería el nivel de destrucción de una casa, a partir de sus características, en caso de que ocurriese una catástrofe del mismo calibre.

Para ello, pasamos a estudiar nuestro dataset Ritcher's Predictor en el que analizaremos las variables incluídas y la naturaleza del propio conjunto de nuestros datos. Después, trataremos de solucionar los problemas que puedan tener las variables de nuestro conjunto de datos y finalemente lo prepararemos para realizar la mejor predicción posible del nivel de daño causado a un edificio por el terremoto de Gorkha.


In [ ]:
from IPython.display import Image, HTML
Image(url='https://s3.amazonaws.com/drivendata-public-assets/nepal-quake-bm-2.JPG')

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import xgboost as xgb


from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import StandardScaler, RobustScaler
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier, StackingClassifier
from sklearn.pipeline import make_pipeline
from sklearn import svm
from sklearn.model_selection import GridSearchCV, train_test_split, RandomizedSearchCV
from sklearn.metrics import f1_score, confusion_matrix, classification_report, accuracy_score, precision_score
from matplotlib.pyplot import figure
from sklearn.pipeline import Pipeline
from imblearn.pipeline import make_pipeline as imb_make_pipeline
from xgboost import XGBClassifier
from scipy.stats.mstats import winsorize


%matplotlib inline
plt.style.use('seaborn')
pd.set_option("display.max_columns",None)
pd.set_option("display.max_rows",None)

## 1. Introducción y presentación de datos

Como ya hemos comentado anteriormente, nuestro cometido en este trabajo es el de preparar un modelo que nos haga predecir de la forma más precisa la categoría de daño que puede sufrir una casa a partir de un terremoto como el de Gorkha en 2015. Podemos realizar esta operación ya que tenemos dos tipos de 'datasets' con los que trabajar; Por un lado, trabajaremos con nuestro conjunto de entrenamiento que llamaremos 'data', en el que nos indican las caracteríticas de cada edificio y su categoría de daño, y después tendremos nuestro conjunto 'test' que será el que utilizaremos para predecir cuál es el nivel de daño causado. 

In [ ]:
# Cargamos los dos conjuntos de datos y la variable respuesta.

data = pd.read_csv('train_values.csv')
test = pd.read_csv('test_values.csv')
train_labels = pd.read_csv('train_labels.csv')


In [ ]:
data.head()

In [ ]:
print(data.shape)
print(train_labels.shape)

En nuestro conjunto de datos de entrenamiento tenemos 39 features, contando con nuestra variable de identificación 'building_id'. Juntamos nuestro dataframe con nuestra variable objetivo 'damage_grade' para poder trabajar en un mismo dataframe.

In [ ]:
# Lo unimos en nuestro dataset de entrenamiento

data = data.merge(train_labels, how = 'inner' , on = 'building_id')

In [ ]:
print(data.shape)

Pasamos a comprobar que nuestra base de datos no disponga de valores nulos o duplicados. Realizamos la comprobación con una tabla inormativa que contenga el tipo de datos de cada variable, la cantidad de valores nulos que incluye y cuantos valores unicos tienen.


In [ ]:
# Tabla informatriva acerca del tipo (Int,object)

tipo_datos=pd.DataFrame(data.dtypes,columns=["Type"])

tipo_datos["Unique"]=data.nunique()
tipo_datos["Null"]=data.isnull().sum()
tipo_datos["% null"]=data.isnull().sum()/len(data)

tipo_datos.style.background_gradient(cmap='Set3',axis=0)


Como nos enseña la tabla que hemos construído, no obtenemos valores faltantes ni duplicados en 'building_id'.

## 2. Analisis variables cuantitativas

Queremos estudiar primero de todo a nuestra variable objetivo 'damage_grade', así que con un histograma podemos ver que categorías son las que más se dan. Después nos ocuparemos del resto de variables cuantitativas y observaremos si tienen problemas tales como valores outliers, mala codificación,etc.

In [ ]:
# Representamos histograma de nuestra variable objetivo 'damage_grade'

plt.figure(figsize = (11,7))
ax = sns.countplot('damage_grade',data = data)

for i in ax.patches :
    ax.text(i.get_x() + 0.3, i.get_height() + 3, \
            str(round((i.get_height()),2)), fontsize = 13 )
plt.title('Damage_grade')
plt.show()


Como muestra el histograma del código anterior, la categoria de dañado medio es la que más datos tiene en nuestro conjunto de entrenamiento, por lo que con esta información podemos ser conscientes del comportamiento de otras variables. Pasamos ahora a observar las variables numéricas de nuestro dataset, concretamente las de localización.

In [ ]:
# Generamos listas de variables cuantitativas que tengan algo en común. Juntamos las de localización por una parte y las de condiciones físicas del edificio por otra.

gps_features = ['geo_level_1_id', 'geo_level_2_id', 'geo_level_3_id']

area_height = ['area_percentage' , 'height_percentage']


Para fijarnos en la distribucón de las variables de localización que las hemos incluido en una lista llamada 'gps_features', nos será de gran utilidad observar la dispersión que muestran las tres variables por cada grupo de nuestra variable respuesta 'damage_grade'.

In [ ]:
# Boxplot de las tres variables de localización (geo_level_1_id, geo_level_2_id, geo_level_3_id)

plt.figure(figsize = (18,18))
q = 1

for i in gps_features:
    plt.subplot(3,3,q)
    ax = sns.boxplot('damage_grade', i, data = data)
    plt.xlabel('damage_grade')
    plt.ylabel(i)
    plt.title('Damage_grade per region' + str(i))
    q += 1
    plt.show
    


Si nos fijamos en nuestros diagramas de cajas para nuestras tres variables de localización de las infraestructuras dañadas, lo primero que observamos es que en geo_level_3 tenemos una variabilidad entre los tres grupos de daño mucho mas baja, ya que en esta variable la localización es más concreta.
En lo que se refiere a los sectores 1-30, vemos que la mayor dispersión se encuentra en el grupo de daño menor, ya que tenemos muchas infraetsructuras dañadas levemente. Este efecto puede ser consecuencia de que el terremoto en Gorkha dañó levemente a sitios muy distanciados, por lo que es difícil que esta variable pueda precisar de forma correcta este grupo.

Representamos gráficamente la distribución de height_percentage y area_percentage, así podemos ver si el comportamiento de esta variable es muy diferente en cada grupo de la variable objetivo. De esta forma, podremos saber si el area o la altura de las infraestructuras dañadas son una característica que afecte al dañado provocado por el terremoto.

In [ ]:
# Representación distirbución area_percentage.

plt.figure(figsize=(20,20))

sns.FacetGrid(data,hue='damage_grade',height=5)\
    .map(sns.distplot,'area_percentage')\
    .add_legend()

plt.title("Area Percentage")
plt.show()


In [ ]:
# Representación distribución 'height_percentage'.

plt.figure(figsize=(14,8))
sns.FacetGrid(data,hue='damage_grade',height=5)\
    .map(sns.distplot,'height_percentage')\
    .add_legend()

plt.title("Height Percentage")
plt.show()

Aunque no podamos extarer conclusiones de momento, podemos observar que tanto la altura del edificio como su area pueden influir perfectamente al nivel de destrucción del edificio después del terremoto. Podemos realizarnos la misma pregunta para la variable 'age', ya que tiene bastante sentido pensar que si el edificio es más antiguo resistió menos el efecto del terremoto.

In [ ]:
# Representación de 'edad' histograma.

plt.figure(figsize=(12,6))
sns.countplot(data['age'], hue=data["damage_grade"])

plt.ylabel("Frequency")
plt.xlabel("Age")
plt.xticks(rotation=90)

plt.title("Age by damaged grade")
plt.legend(["Low","Medium","High"])
plt.show()
    

Vemos que la gran mayoría de infraestructuras dañadas tienen alrededor de 5 a 20 años, la gran mayoría de ellas clasificadas con el nivel medio de destrucción. También se muestra que la categoría de daño más bajo se reduce exponencialmente a medida que los edificios son más antiguos, por lo que estos ya reciben un daño medio o destrucción total. Al mismo tiempo, la evolución en términos proporcionales de la categoría 'High' va ganando terreno a 'Medium' a medida que aumenta la edad. Finalmente, eliminaremos más adelante los valores atípicos que encontramos en la variable 'age', ya que se encuentran totalmente alejados de nuestra distribución de datos y pueden no ser representativos. 

In [ ]:
# Histograma de 'count_floors_pre_eq'

plt.figure(figsize=(12,6))
sns.countplot(data['count_floors_pre_eq'], hue=data["damage_grade"])

plt.ylabel("Frequency")
plt.xlabel("count_floors_pre_eq")
plt.xticks(rotation=90)

plt.title("Count_floors_pre_eq")
plt.legend(["Low","Medium","High"])
plt.show()
    

En el caso de count_floors_pre_eq se muestra que la categoria de daño 'High' se encuentra más presente en términos proporcionales cuando el edificio en cuestion pasa de tener una planta a tres. Es lógico pensar que esta variable se encuentre correlacionada con 'height_percentage' ya que esta última tambien podria tener un impacto positivo en el daño causado al edificio. De esta manera, count_floors_pre_eq nos podría ser de gran ayuda cuando entrenemos un modelo de predicción.

## 3. Analisis variables categóricas y binarias

Una vez hemos analizado las distribuciones de nuestras variables cuantitativas y ver cuales presentan algunos problemas y si nos podrían ser de interés, podemos hacer lo mismo con las variables categóricas representando su histograma y ver como se comportan las categorías de 'damage_grade'.

In [ ]:
# Creamos lista de categóricas 

categoricas = ['plan_configuration','land_surface_condition','foundation_type','roof_type',
              'ground_floor_type', 'other_floor_type','position','legal_ownership_status']

# Realizamos un encuadre de histogramas con nuestras variables categóricas mediante seaborn.
    
q=1
plt.figure(figsize=(20,20))

for j in categoricas:
    plt.subplot(3,3,q)
    ax=sns.countplot(data[j].dropna(), hue=data["damage_grade"])
    plt.xlabel(j)
    plt.legend(["Low ","Medium","High"])
    q+=1
    
plt.show()


Analizamos, de la misma forma que lo hemos hecho con las variables categóricas, las variables binarias que hacen referencia a la superestructura de los edificios y al de uso secundario de las infraestructuras dañadas. Estos gráficos nos dejan algunas ideas que podemos tomar en consideración que se encuentran explicadas en nuestro informe. 

In [ ]:
# Seleccionamos las variables binarias que hacen referencia a la superestructura de los edificios.

superstructure_cols = ['has_superstructure_adobe_mud' , 'has_superstructure_mud_mortar_stone' , 'has_superstructure_stone_flag' ,
                      'has_superstructure_cement_mortar_stone', 'has_superstructure_mud_mortar_brick' , 'has_superstructure_cement_mortar_brick'
                      , 'has_superstructure_timber' , 'has_superstructure_bamboo' , 'has_superstructure_rc_non_engineered'
                      , 'has_superstructure_rc_engineered' , 'has_superstructure_other']    

q=1
plt.figure(figsize=(20,20))

# Por cada variable en la lista, generar un histograma e incluirlo a nuestro subplot.

for j in superstructure_cols:
    plt.subplot(4,4,q)
    ax=sns.countplot(data[j].dropna(), hue=data["damage_grade"])
    plt.xlabel(j)
    plt.legend(["Low ","Medium","High"])
    q+=1
    
plt.show()


Realizamos la misma operación para las variables binarias de uso secundario.

In [ ]:
# Seleccionamos las variables binarias restantes para observar los histogramas.

secondary_use = ['has_secondary_use', 'has_secondary_use_agriculture' , 'has_secondary_use_hotel' , 'has_secondary_use_rental', 
                    'has_secondary_use_institution' , 'has_secondary_use_school' , 'has_secondary_use_industry', 
                    'has_secondary_use_health_post','has_secondary_use_gov_office' , 'has_secondary_use_use_police' , 
                    'has_secondary_use_other' ]

q=1
plt.figure(figsize=(20,20))

# Recorremos la lista para ir incluyendo los histogramas en el subplot generado.

for j in secondary_use:
    plt.subplot(4,4,q)
    ax=sns.countplot(data[j].dropna(), hue=data["damage_grade"])
    plt.xlabel(j)
    plt.legend(["Low ","Medium","High"])
    q+=1
    
plt.show()

## 4. Preprocesamiento de datos y valores outliers

Después de haber representado y comentado de forma más extensa en el informe las propiedades de las variables independientes y objetivo, es necesario volver a las variables númericas para solucionar el problema de valores outliers que habíamos detectado en variables como por ejemplo 'age'. Para ello, realizamos el proceso winsorized para poder ajustar la distribución de las observaciones extremas. Este proceso se realiza para las variables 'age', 'height_percentage' y 'area_percentage', de manera que ya estarían listas para poder analizar las correlaciones con otras variables (analisis multivariante) y para incluirlas en un modelo de predicción.

In [ ]:
# Outilers en variables numéricas.

num_cols=["age","area_percentage","height_percentage"]
q=1
plt.figure(figsize=(20,20))

for j in num_cols:
    plt.subplot(3,3,q)
    ax=sns.boxplot(data[j].dropna(), palette="colorblind")
    plt.xlabel(j)
    q+=1

plt.show()

Disponemos de valores outliers en las variables que hemos incluído en los gráficos de caja. Deberemos arreglar el problema de los outliers de manera que el efecto de estas variables sobre nuestra variable respuesta no se desvirtualize debido a los valores extremos que observamos. Mostramos una representación del diagrama de cajas de estas tres variables numéricas antes y después del proceso 'winsorized' para comparar como hemos arreglado la dispersión de la variable.

In [ ]:
# Seleccionamos la variable 'age' para realizar la estabilicación de los valores outliers que encontramos.

n = 'age'
target = data[n]

# Dibujamos un boxplot para comparar "winsonorization process" de nuestros valores outliers.

sns.boxplot(target)
plt.title("{} No Winsorization".format(n))
plt.show()

# Realizamos winsonorization y comparamos con el gráfico anterior.

winsorized_target = winsorize(target,(0,0.05))

# Boxplot winsorized

sns.boxplot(winsorized_target)
plt.title("{} Winsorized". format(n))
plt.show

# Las pasamos a nuestro dataset de entrenamiento.

data['age'] = winsorized_target


Vemos que tenemos una distribución de nuestra variable edad mucho más balanceada al transformar los valores outliers que teníamos anteriormente. En este momento, la centralidad de la variable edad se encuentra entre los 10 y 30 años, con una media de 21 años, antes la teníamos alrededor de los 26 años de edad. 


In [ ]:
# Seleccionamos la variable 'area_percentege' para realizar la estabilicación de los valores outliers que encontramos.

n = 'area_percentage'
target = data[n]

# Dibujamos un boxplot para comparar "winsonorization process" de nuestros valores outliers.

sns.boxplot(target)
plt.title("{} No Winsorization".format(n))
plt.show()

# Realizamos winsonorization y comparamos con el gráfico anterior.

winsorized_target = winsorize(target,(0,0.04))

# Boxplot winsorized

sns.boxplot(winsorized_target)
plt.title("{} Winsorized". format(n))
plt.show

# Las pasamos a nuestro dataset de entrenamiento

data['area_percentage'] = winsorized_target

In [ ]:
# Seleccionamos la variable 'height_percentage' para realizar la estabilicación de los valores outliers que encontramos.

n = 'height_percentage'
target = data[n]

# Dibujamos un boxplot para comparar "winsonorization process" de nuestros valores outliers.

sns.boxplot(target)
plt.title("{} No Winsorization".format(n))
plt.show()

# Realizamos winsonorization y comparamos con el gráfico anterior.

winsorized_target = winsorize(target,(0,0.04))

# Boxplot winsorized

sns.boxplot(winsorized_target)
plt.title("{} Winsorized". format(n))
plt.show

# Las pasamos a nuestro dataset de entrenamiento

data['height_percentage'] = winsorized_target

Pasamos a guardar las features que queremos incluir en nuestro modelo en un nuevo dataframe, tambien realizaremos un gráfico de correlaciones entre las variables incluidas para ver si podemos prescindir de algunas más en nuestro modelo.

## 5. Analisis correlaciones

Cuando ya hemos preprocesado las variables que presentaban tener algun problema que pudiera distorsionar la relación que puediesen tener con otras variables o con la propia variable respuesta, estamos listos para crear una matriz de correlaciones y ver de esta manera cuales son las variables que parecen tener relaciones y cuales pueden ser determinantes sobre la variable objetivo 'damage_grade'.

In [ ]:
# Nuevo dataset donde tenemos simplemente las variables independientes de nuestro conjunto de datos original.

features = data.copy()
features = features.drop(['building_id'], axis = 1)

features.head()

In [ ]:
# Generamos una matriz de correlaciones con las features que tenemos incluídas.

corr_matrix = features.corr()

# Diseñamos un mapa de calor para representar visualmente la matriz de correlaciones.

mask = np.triu(np.ones_like(corr_matrix, dtype=bool))

f, ax = plt.subplots(figsize = (14,14))

cmap = sns.diverging_palette(230, 20, as_cmap=True)

sns.heatmap(corr_matrix, mask=mask, cmap=cmap, vmax=1, vmin = -1, center=0,
            square=True, linewidths=.5, cbar_kws={"shrink": .5})

plt.show()


A partir del mapa de calor generado en el código anterior podemos ver cuales son las variables que se encuentran más correlacionadas, ya sea de positiva o negativamente.

Las variables que más correlacionadas se encuentran con el tipo de daño causado son las que hacen referencia a la superestructura de los edificios,concretamente,'has_superstrcuture_cement_mortar_brick','has_superstructure_rc_engineered', 'has_superstructure_rc_non_engineered' y 'has_superstructure_mud_mortar_stone'. También hemos comentado anteriormente que la edad es una variable que parece tener un efecto causal con el daño causado en el edificio. Después encontramos variables con una gran correlacion como 'count_floors_pre_eq' con 'height_percentage', o 'has_secondary_use' con 'has_secondary_use_agriculture'.




Observamos las feautures que tienen una mayor correlación con nuestra variable otuput 'damage_grade'.

In [ ]:
# Mapa de calor para ver las correlaciones de 'damage_grade'.

plt.figure(figsize = (10,10))
correlations = features.corrwith(features['damage_grade'])
correlations = correlations.sort_values(ascending = False)
sns.heatmap(pd.DataFrame(correlations), annot = True)
plt.show()

Por lo que sabemos del comportamiento de variables y por lo que hemos visto anteriormente en nuestro correlograma, no es muy recomendable incluir variables que esten muy correlacionadas entre si y que al mismo tiempo no tengan un impacto notable en la variable que queremos predecir. Por ello, trataremos de eliminar de nuestro conjunto de features las variables 'has_secondary_use_agriculture' y has_superstructure_bamboo', ya que no nos serán de gran utilidad en nuestra predicción.

In [ ]:
# Eliminamos features con mucha correlación con otras que son más importantes.

features = features.drop(['has_secondary_use_agriculture', 'has_superstructure_bamboo'], axis = 1)


## 6. Entrenamiento y evaluación 

Llegamos a la parte más importante del trabajo. Con nuestro dataset preparado para entrenar un modelo, nos quedamos solo con las features que nos permitan realizar la predicción de la forma más precisa posible. Eliminamos 'damage_grade' al ser la variable respuesta, pasamos a tipo 'category' las variables categóricas y después realizamos la partición de los datos para medir la fiabilidad del modelo que eligamos.

In [ ]:
# Generamos el dataframe X que emula a nuestro dataset original, de manera que nos servirá para realizar la predicción.
X = features

X = X.drop(['damage_grade'], axis = 1)

# Guardamos nuestra variable output en una lista.
y = features['damage_grade']


In [ ]:
X = pd.get_dummies(X)

Una vez tenemos nuestro conjunto de datos X proveniente de nuestro training set, pasamos a realizar la partición de nuestro dataset para testear la precisión de nuestra predicción. Finalmente, entrenaremos algunos modelos que nos sean candidatos para realizar la predicción final de nuestro conjunto de testeo, los iremos evaluando en función de la f1_score que obtengan en cada entrenamiento.

In [ ]:
X_train, X_test,y_train, y_test = train_test_split(X,y,test_size = 0.2,random_state = 42)

### Random Forest Model

Con nuestros dos conjuntos de datos listos para la estimación, empezamos realizando un Random Forest Classifier después de buscar los hiperparámetros claves para la predicción. Despúes observemos la matriz de confusión de nuestro modelo y estudiaremos los resultados en base a la medida f1_score que es en la que se basa la competición.

In [ ]:

"""
rf = RandomForestClassifier()

param_grid = {'n_estimators': [100,150,200,250,300,350,400],'max_depth': [None,10,20,25,30,35,40,45,50]}

grid_search_rf = RandomizedSearchCV(rf, param_distributions=param_grid, n_iter=15, 
                                    cv=5, n_jobs=6, verbose=1)

grid_search_rf.fit(X_train,y_train)

print(pd.DataFrame(grid_search_rf.cv_results_).sort_values(by='rank_test_score').head())
grid_search_rf.score(X_test,y_test)

"""

In [ ]:
# Realizamos el modelo Random Forest con los hiperparámteros obtenidos

rf = RandomForestClassifier(max_features = None,
                            max_depth = 30,
                            n_estimators= 150,
                            min_samples_split = 3,
                            min_samples_leaf = 30,
                            random_state = 42)

# Lo aplicamos a nuestro conjuntop train y realizamos la predicción sobre el conjunto test.

rf.fit(X_train,y_train)
rf_pred=rf.predict(X_test)

# Diseñamos una matriz de confusión para ver donde ha cometido más errores la predicción del modelo.

cm=confusion_matrix(y_test,rf_pred)
conf_matrix=pd.DataFrame(data=cm,columns=['Predicted:1','Predicted:2','Predicted:3'],
                                         index=['Actual:1','Actual:2','Actual:3'])
              
# Representamos la matriz gráficamente.
    
plt.figure(figsize = (8,5))
sns.heatmap(conf_matrix, annot=True,fmt='d',cmap="YlGnBu")
plt.title("confusion Matrix for  Random Forest")
plt.xticks(rotation=45)
plt.yticks(rotation=360)
plt.show()


In [ ]:
# Mostramos la precisión del modelo y classification report.

print('-'*100)
print('Accuracy on Random Forest', accuracy_score(y_test,rf_pred))
print('-'*100)
print("\n")
print("Classification Report :\n\n " , classification_report(y_test,rf_pred))
print("-"*100)


De momento, tal y como nos muestra nuestra matriz de confusión, la predicción Random Forest se ajusta bastante bien a los datos observados en y_test. Además obtenemos una accuracy de 0.731, la cual es una buena puntuación comparada con la media que observamos en la competición DrivenData. Veremos si podemos mejorarla con otras alternativas.

### Boosting Classifier

Hemos visto que el modelo Random Forest funciona con bastante precisión, pero debemos probar con otros modelos que nos permitan obtener una accuracy_score mayor para poder realizar la estimación final out-of-sample.

In [ ]:
# Definimos nuestro modelo de predicción boosting con objetivo softmax al tener tres clases de objetivo

xgb = XGBClassifier(objective = 'multi:softmax',
                    num_class = 3,
                    random_state = 42,
                    max_depth = 20,
                    n_estimators = 150,                    
                    )

xgb.fit(X_train,y_train)
xgb_pred = xgb.predict(X_test)

# Definimos nuestra matriz de confusión y la representamos gráficamente

cm = confusion_matrix(y_test,xgb_pred)
conf_matrix=pd.DataFrame(data=cm,columns=['Predicted:1','Predicted:2','Predicted:3'],
                                         index=['Actual:1','Actual:2','Actual:3'])



plt.figure(figsize = (8,5))
sns.heatmap(conf_matrix, annot=True,fmt='d',cmap="YlGnBu")
plt.title("Confusion Matrix for Boosting Classifier")
plt.xticks(rotation=45)
plt.yticks(rotation=360)
plt.show()


In [ ]:
print('-'*100)
print('Accuracy on XGBoosting', accuracy_score(y_test,xgb_pred))
print('-'*100)
print("\n")
print("Classification Report :\n\n " , classification_report(y_test,xgb_pred))
print("-"*100)


El modelo XGBoosting se adapta bien a nuestros datos pero obtiene una medida de precisión más baja que nuestro primer modelo, por lo que Random Forest sería más útil para realizar una predicción. Sin embargo, vemos que XGBoosting clasifica mejor las clases de daño bajo y alto, por lo que podríamos utilizar este modelo para fusionarlo con nuestro Random Forest. De esta manera, probamos con un modelo final que combine los dos que hemos entrenado anteriormente mediante StackingClassifier. Finalmente utilizamos en este modelo una regresión logística como clasificador final para el modelo que combina RandomForest y XGBoosting.

### Ensamble model

In [ ]:
# Definimos un modelo de ensamble con los dos anteriores que hemos entrenado.

models = [
     ("rf",rf),("xgb", xgb)
     
]
f_model = StackingClassifier(estimators=models, final_estimator=LogisticRegression(),verbose=1,n_jobs=6)
f_model.fit(X_train, y_train)


final_pred = f_model.predict(X_test)

# Definimos nuestra matriz de confusión y la representamos gráficamente

cm = confusion_matrix(y_test,final_pred)
conf_matrix=pd.DataFrame(data=cm,columns=['Predicted:1','Predicted:2','Predicted:3'],
                                         index=['Actual:1','Actual:2','Actual:3'])


plt.figure(figsize = (8,5))
sns.heatmap(conf_matrix, annot=True,fmt='d',cmap="YlGnBu")
plt.title("Confusion Matrix for Ensamble model")
plt.xticks(rotation=45)
plt.yticks(rotation=360)
plt.show()



In [ ]:
print('-'*100)
print('Accuracy on Ensamble model', accuracy_score(y_test,final_pred))
print('-'*100)
print("\n")
print("Classification Report :\n\n " , classification_report(y_test,final_pred))
print("-"*100)

Tal y como se muestra en la matriz de clasificación y en la precisión de predicción en nuestro conjunto test, el modelo de ensamble final es el que mejor clasifica la variable objetivo damage_grade. Ya que tenemos un modelo ganador, ahora nos toca realizar la predicción out-of-sample con el conjunto test que cargamos al principio del proyecto y guardar nuestras predicciones.

## 7. Predicción out-of-sample

Finalmente, vamos a realizar la predicción final de nuestro modelo con nuestro conjunto test y podremos subir a la competición nuestras predicciones con el modelo final. Sin embargo, antes debemos preprocesar el conjunto test de la misma manera que lo hemos hecho con nuestro conjunto de entrenamiento durante todo el trabajo; Esto incluye eliminar las variables que finalmente no se incluyeron, arreglar el problema de los outliers con winsorized y transformar las variables categóricas a binarias para mejorar la predicción. Después de tener el conjunto de datos preparado ya solo nos queda realizar la predicción final de nuestro conjunto test y guardar la predicción con el formato requerido de la competición de DrivenData para que nos pueda calcular nuestra puntuacón de f1_score final.

In [ ]:
# Variables eliminadas.

test = test.drop(['building_id','has_secondary_use_agriculture','has_superstructure_bamboo'], axis = 1)

# Outliers.

n = 'age'
target = test[n] 
winsorized_age = winsorize(target,(0,0.05))
test['age'] = winsorized_age


n = 'area_percentage'
target = test[n] 
winsorized_area = winsorize(target,(0,0.04))
test['area_percentage'] = winsorized_area


n = 'height_percentage'
target = test[n] 
winsorized_height = winsorize(target,(0,0.04))
test['height_percentage'] = winsorized_height

# Pasamos las variables categóricas a dummies.

test = pd.get_dummies(test)


In [ ]:
# Realizamos la predicción final

prediction = f_model.predict(test)

Generamos un nuevo dataframe donde se incluya la identificación de las infraestructuras del conjunto test y el 'damage_grade' que ha predecido nuestro algoritmo. 

In [ ]:
# Cargamos el formato de subida que nos requiere DrivenData

my_submission = pd.read_csv('submission_format.csv')
my_submission = my_submission.drop('damage_grade', axis = 1)

# Subimos nuestra predicción.

prediction = pd.DataFrame(prediction)
my_submission['damage_grade'] = prediction
my_submission = my_submission.set_index('building_id')

my_submission.head()

In [ ]:
# Guardamos nuestra predicción en un csv.

my_submission.to_csv('my_submission1.csv')

In [ ]:
!head my_submission1.csv

Una vez generado el dataframe con nuestras predicciones para el conjunto test, lo subimos a la competición de DrivenData con el formato que se requiere. Finalmente, la f1_score obtenida en el conjunto test es de 0.7401, lo cual es una gran puntuación ya que nos situamos en el ranking 447 de los 4361 participantes totales. La mejora de nuestra predicción viene dada por el uso de un modelo más óptimo y por el procesamiento que hemos realizado anteriormente con nuestros datos, para ello ha sido imprescindible entender y analizar el comportamiento de nuestras variables.